#### Prerequisites

* Switch instance type to `ml.m5.large`

### I. Imports 

In [ ]:
from sagemaker import Session
from pandas import DataFrame
from time import sleep
import pandas as pd
import sagemaker
import boto3

##### Essentials

In [ ]:
session = Session()
s3 = boto3.resource('s3')

S3_BUCKET = session.default_bucket()
S3_DATA_FOLDER = 'data'

### II. Load and prepare data 

In [ ]:
%%time

df = pd.read_csv('s3://sagemaker-us-east-1-119174016168/datasets/covid19_articles_20220420.csv')

In [ ]:
df.shape

In [ ]:
df.drop(df.filter(regex='Unnamed'), axis=1, inplace=True)

In [ ]:
df.drop(['author', 'date', 'domain', 'companies', 'locations', 'sdgs'], axis=1, inplace=True)

In [ ]:
def clean(article: list) -> str:
    return '\n'.join(eval(article))

In [ ]:
df['content'] = df['content'].apply(clean)

In [ ]:
df.rename(columns={'datatype': 'category'}, inplace=True)

In [ ]:
df.head()

### III. Prepare dataset for BERT MLM training 

In [ ]:
mlm_df = df[['title', 'content']]

In [ ]:
mlm_df.head()

In [ ]:
def save_as_txt(df: DataFrame) -> None:
    fout = open('./data/covid_articles.txt', 'w')
    for title, content in zip(df.title.values, df.content.values):
        fout.write('\n'.join([title, content]))
    fout.close()

In [ ]:
%%time

save_as_txt(df)

In [ ]:
sleep(5)

##### Copy dataset from local to S3

In [ ]:
s3.meta.client.upload_file('./data/covid_articles.txt', S3_BUCKET, f'{S3_DATA_FOLDER}/covid_articles.txt')

### IV. Prepare dataset for sequence classification 

In [ ]:
clf_df = df.dropna()

In [ ]:
def combine(title: str, content: str) -> str:
    return '\n'.join([title, content])

In [ ]:
clf_df['article'] = clf_df.apply(lambda x: combine(x['title'], x['content']), axis=1)

In [ ]:
clf_df.drop(['title', 'content'], axis=1, inplace=True)

In [ ]:
clf_df.head()

In [ ]:
assert len(set(clf_df.category.values)) == 5

##### Copy dataset from local to S3 

In [ ]:
clf_df.to_csv('./data/covid_articles_clf_data.csv', index=False)

In [ ]:
sleep(5)

In [ ]:
s3.meta.client.upload_file('./data/covid_articles_clf_data.txt', S3_BUCKET, f'{S3_DATA_FOLDER}/covid_articles_clf_data.txt')